In [76]:
import pandas as pd
import matplotlib as plt
import numpy as np
import requests

In [77]:
url = "../data/raw_data.csv"

In [78]:
df = pd.read_csv(url)
df

,Entity,Code,Year,"Meat of cattle with the bone, fresh or chilled | 00000867 || Producing or slaughtered animals | 005320 || animals","Meat, goat | 00001017 || Producing or slaughtered animals | 005320 || animals","Meat, chicken | 00001058 || Producing or slaughtered animals | 005321 || animals","Meat, turkey | 00001080 || Producing or slaughtered animals | 005321 || animals","Meat, pig | 00001035 || Producing or slaughtered animals | 005320 || animals","Meat, lamb and mutton | 00000977 || Producing or slaughtered animals | 005320 || animals","Meat, duck | 00001069 || Producing or slaughtered animals | 005321 || animals"
0,Afghanistan,AFG,1961,360000.0,940000.0,7000000.0,NaN,NaN,4336000.0,NaN
1,Afghanistan,AFG,1962,384000.0,875000.0,7500000.0,NaN,NaN,4355000.0,NaN
2,Afghanistan,AFG,1963,396000.0,810000.0,7700000.0,NaN,NaN,4673000.0,NaN
3,Afghanistan,AFG,1964,402000.0,750000.0,8000000.0,NaN,NaN,5010000.0,NaN
4,Afghanistan,AFG,1965,408000.0,875000.0,8500000.0,NaN,NaN,5179000.0,NaN
...,...,...,...,...,...,...,...,...,...,...
14575,Zimbabwe,ZWE,2019,2350000.0,2461470.0,71479000.0,29000.0,193820.0,51842.0,28000.0
14576,Zimbabwe,ZWE,2020,2300000.0,2501662.0,71367000.0,30000.0,183923.0,52614.0,28000.0
14577,Zimbabwe,ZWE,2021,2576748.0,2587830.0,82885000.0,30000.0,196173.0,47741.0,26000.0
14578,Zimbabwe,ZWE,2022,2675482.0,2713257.0,69773000.0,31000.0,283905.0,48159.0,25000.0


In [79]:
df.columns

Index(['Entity', 'Code', 'Year',
       'Meat of cattle with the bone, fresh or chilled | 00000867 || Producing or slaughtered animals | 005320 || animals',
       'Meat, goat | 00001017 || Producing or slaughtered animals | 005320 || animals',
       'Meat, chicken | 00001058 || Producing or slaughtered animals | 005321 || animals',
       'Meat, turkey | 00001080 || Producing or slaughtered animals | 005321 || animals',
       'Meat, pig | 00001035 || Producing or slaughtered animals | 005320 || animals',
       'Meat, lamb and mutton | 00000977 || Producing or slaughtered animals | 005320 || animals',
       'Meat, duck | 00001069 || Producing or slaughtered animals | 005321 || animals'],
      dtype='object')

In [80]:
df.columns = df.columns[:3].tolist() + [
    'Cattle', 'Goat', 'Chicken', 'Turkey', 'Pig', 'Lamb/Mutton', 'Duck'
]
df.columns

Index(['Entity', 'Code', 'Year', 'Cattle', 'Goat', 'Chicken', 'Turkey', 'Pig',
       'Lamb/Mutton', 'Duck'],
      dtype='object')

In [81]:
unique_entities = df['Entity'].unique()
unique_entities

array(['Afghanistan', 'Africa', 'Africa (FAO)', 'Albania', 'Algeria',
       'Americas (FAO)', 'Angola', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Asia', 'Asia (FAO)', 'Australia', 'Austria',
       'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belgium-Luxembourg (FAO)', 'Belize',
       'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Caribbean (FAO)',
       'Central African Republic', 'Central America (FAO)',
       'Central Asia (FAO)', 'Chad', 'Chile', 'China', 'China (FAO)',
       'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Czechoslovakia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor',
       'Eastern Africa (FAO)', 'Eastern Asia (

In [82]:
country_entities = [i for i in unique_entities if "(" not in i]
country_entities

['Afghanistan',
 'Africa',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Asia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Czechoslovakia',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Europe',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Guiana',
 'French Polynesia',
 'Gabon',
 'Gambia',
 'Georgia',
 'Ge

In [83]:
paran_entities = [i for i in unique_entities if "(" in i]
print(paran_entities)
print(len(paran_entities))

['Africa (FAO)', 'Americas (FAO)', 'Asia (FAO)', 'Belgium-Luxembourg (FAO)', 'Caribbean (FAO)', 'Central America (FAO)', 'Central Asia (FAO)', 'China (FAO)', 'Eastern Africa (FAO)', 'Eastern Asia (FAO)', 'Eastern Europe (FAO)', 'Ethiopia (former)', 'Europe (FAO)', 'European Union (27)', 'European Union (27) (FAO)', 'Land Locked Developing Countries (FAO)', 'Least Developed Countries (FAO)', 'Low Income Food Deficit Countries (FAO)', 'Micronesia (FAO)', 'Micronesia (country)', 'Middle Africa (FAO)', 'Net Food Importing Developing Countries (FAO)', 'Northern Africa (FAO)', 'Northern America (FAO)', 'Northern Europe (FAO)', 'Oceania (FAO)', 'Small Island Developing States (FAO)', 'South America (FAO)', 'South-eastern Asia (FAO)', 'Southern Africa (FAO)', 'Southern Asia (FAO)', 'Southern Europe (FAO)', 'Sudan (former)', 'Western Africa (FAO)', 'Western Asia (FAO)', 'Western Europe (FAO)']
36


**'China (FAO)'**,   **'Micronesia (FAO)'**,   **'Sudan (former)'**,  **'Micronesia (country)'**,  **'Sudan (former)'**,  and  **'Belgium-Luxembourg (FAO)'**   are  all  countries  and  not  regions

In [84]:
exception_countries = ['China (FAO)','Micronesia (FAO)', 'Micronesia (country)', 'Sudan (former)']

In [85]:
conditions = [
    df['Entity'].isin(exception_countries),
    df['Entity'].str.contains(r'\('),
]
values = [0,1]

In [86]:
df['Region'] = np.select(conditions, values, default=0)
df[['Entity', 'Region']]

,Entity,Region
0,Afghanistan,0
1,Afghanistan,0
2,Afghanistan,0
3,Afghanistan,0
4,Afghanistan,0
...,...,...
14575,Zimbabwe,0
14576,Zimbabwe,0
14577,Zimbabwe,0
14578,Zimbabwe,0


In [87]:
df[conditions[1]]

,Entity,Code,Year,Cattle,Goat,Chicken,Turkey,Pig,Lamb/Mutton,Duck,Region
126,Africa (FAO),NaN,1961,13234469.0,28137736.0,3.584050e+08,1425000.0,3835586.0,37204512.0,8221000.0,1
127,Africa (FAO),NaN,1962,13352172.0,28830964.0,3.729990e+08,1510000.0,3987514.0,37690812.0,8411000.0,1
128,Africa (FAO),NaN,1963,13446035.0,28471024.0,3.866970e+08,1585000.0,4070344.0,37940312.0,8587000.0,1
129,Africa (FAO),NaN,1964,13876142.0,29031172.0,4.093810e+08,1670000.0,4313906.0,39515352.0,8783000.0,1
130,Africa (FAO),NaN,1965,14132494.0,30167680.0,4.282280e+08,1756000.0,4536320.0,40663472.0,8860000.0,1
...,...,...,...,...,...,...,...,...,...,...,...
14292,Western Europe (FAO),NaN,2019,12234833.0,982297.0,2.478662e+09,76031000.0,113744448.0,7041642.0,95756000.0,1
14293,Western Europe (FAO),NaN,2020,11916679.0,979850.0,2.468101e+09,76570000.0,112184000.0,7147191.0,80464000.0,1
14294,Western Europe (FAO),NaN,2021,11858301.0,1013953.0,2.394428e+09,71431000.0,111826576.0,7168472.0,72798000.0,1
14295,Western Europe (FAO),NaN,2022,11408345.0,1014744.0,2.365136e+09,63106000.0,105195712.0,6920516.0,54161000.0,1


In [88]:
meat_columns = df.columns[4:]  # All meat types

df_long = df.melt(
    id_vars=['Entity', 'Code', 'Year', 'Region'],
    value_vars=meat_columns,
    var_name='Meat_Type',
    value_name='Slaughter_Count'
)
df_long

,Entity,Code,Year,Region,Meat_Type,Slaughter_Count
0,Afghanistan,AFG,1961,0,Goat,940000.0
1,Afghanistan,AFG,1962,0,Goat,875000.0
2,Afghanistan,AFG,1963,0,Goat,810000.0
3,Afghanistan,AFG,1964,0,Goat,750000.0
4,Afghanistan,AFG,1965,0,Goat,875000.0
...,...,...,...,...,...,...
87475,Zimbabwe,ZWE,2019,0,Duck,28000.0
87476,Zimbabwe,ZWE,2020,0,Duck,28000.0
87477,Zimbabwe,ZWE,2021,0,Duck,26000.0
87478,Zimbabwe,ZWE,2022,0,Duck,25000.0


In [89]:
resp = requests.get("https://restcountries.com/v3.1/all?fields=name,flags")
countries = resp.json()
countries

[{'flags': {'png': 'https://flagcdn.com/w320/md.png',
   'svg': 'https://flagcdn.com/md.svg',
   'alt': 'The flag of Moldova is composed of three equal vertical bands of blue, yellow and red, with the national coat of arms centered in the yellow band.'},
  'name': {'common': 'Moldova',
   'official': 'Republic of Moldova',
   'nativeName': {'ron': {'official': 'Republica Moldova',
     'common': 'Moldova'}}}},
 {'flags': {'png': 'https://flagcdn.com/w320/km.png',
   'svg': 'https://flagcdn.com/km.svg',
   'alt': 'The flag of Comoros is composed of four equal horizontal bands of yellow, white, red and blue, with a green isosceles triangle superimposed on the hoist side of the field. This triangle has its base on the hoist end, spans about two-fifth the width of the field and bears a fly-side facing white crescent and four five-pointed white stars arranged in a vertical line along the opening of the crescent.'},
  'name': {'common': 'Comoros',
   'official': 'Union of the Comoros',
   'n

In [90]:
flags_list = [
    {
      "Entity": c.get("name", {}).get("common"),
      "Flag_URL": c.get("flags", {}).get("png") or c.get("flags", {}).get("svg")
    }
    for c in countries
]
flags_df = pd.DataFrame(flags_list)
flags_df

,Entity,Flag_URL
0,Moldova,https://flagcdn.com/w320/md.png
1,Comoros,https://flagcdn.com/w320/km.png
2,Malaysia,https://flagcdn.com/w320/my.png
3,Algeria,https://flagcdn.com/w320/dz.png
4,Norfolk Island,https://flagcdn.com/w320/nf.png
...,...,...
245,Dominican Republic,https://flagcdn.com/w320/do.png
246,Belgium,https://flagcdn.com/w320/be.png
247,Mozambique,https://flagcdn.com/w320/mz.png
248,Mauritius,https://flagcdn.com/w320/mu.png


In [91]:
db_flag_names = flags_df['Entity']
db_flag_names

0                 Moldova
1                 Comoros
2                Malaysia
3                 Algeria
4          Norfolk Island
              ...        
245    Dominican Republic
246               Belgium
247            Mozambique
248             Mauritius
249            Azerbaijan
Name: Entity, Length: 250, dtype: object

In [92]:
all_countries_of_slaught = country_entities + exception_countries
all_countries_of_slaught

['Afghanistan',
 'Africa',
 'Albania',
 'Algeria',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Asia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Czechoslovakia',
 'Democratic Republic of Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'East Timor',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Europe',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Guiana',
 'French Polynesia',
 'Gabon',
 'Gambia',
 'Georgia',
 'Ge

In [93]:
left_out_flags = []

# Normalize database names: lowercase, stripped
db_flag_names_normalized = set(name.lower().strip() for name in db_country_names)

for i in all_countries_of_slaught:
    if '(' in i:
        cleaned = i.split('(')[0].strip().lower()
    else:
        cleaned = i.strip().lower()

    if cleaned not in db_flag_names_normalized:
        left_out_flags.append(i)

print(f"{len(left_out_flags)} countries without matching flag")
print(left_out_flags)  # preview some of them


24 countries without matching flag
['Africa', 'Asia', 'Congo', "Cote d'Ivoire", 'Czechoslovakia', 'Democratic Republic of Congo', 'East Timor', 'Europe', 'High-income countries', 'Low-income countries', 'Lower-middle-income countries', 'Macao', 'Melanesia', 'North America', 'Oceania', 'Polynesia', 'Reunion', 'Sao Tome and Principe', 'Serbia and Montenegro', 'South America', 'USSR', 'Upper-middle-income countries', 'World', 'Yugoslavia']


## 🧠 Country & Region Classification – Data Challenges & Realizations

While preparing this dataset for visualization in Power BI, I needed to classify each `Entity` as either a **country** or a **region**. Initially, I used a shortcut approach:

> If an entity contained parentheses and wasn't in a known exception list, it was considered a region. Otherwise, it was a country.

### 🔍 What I Discovered

- ✅ This shortcut worked *mostly*, but not perfectly.
- ❌ Some **countries** did **not contain parentheses** and were **missing from the web-scraped country list**, leading to false mismatches:
  - `Congo`
  - `Democratic Republic of Congo`
  - `Cote d'Ivoire`
  - `East Timor`
  - `Sao Tome and Principe`
  - `Czechoslovakia` *(historic, not in modern APIs)*

  ➤ ✅ These were manually corrected and flagged as valid **countries**.

- ❌ Some **regions** had **no parentheses** in their names, which made the logic misclassify them as countries:
  - `Macao`
  - `Melanesia`
  - `Polynesia`

  ➤ ✅ These were manually reclassified as **regions**.

### ✅ Resolution & Next Steps

- I moved away from string-format-based logic (e.g., checking for `"("`) and instead:
  - Used an external API (`restcountries.com`) to build a list of valid modern countries
  - Normalized and compared cleaned names (`lowercase`, trimmed) across sources
  - Manually corrected a small list of edge cases, both countries and regions

This approach ensures:
- ✅ Accurate classification of entities
- ✅ Proper flag matching for valid countries
- 🛑 Excludes regions from unnecessary processing

---

📌 *This classification step ensures that Power BI dashboards calculate metrics like rankings and comparisons only within the correct category—countries vs. regions—enhancing the reliability and interpretability of the visualizations.*


In [94]:
#df.to_csv('../data/preprocessed_data.csv', index=False)